In [ ]:
import numpy as np 
import pandas as pd 
import glob
import os
import nibabel as nib               # NIfTI format -> .nii files
import matplotlib.pyplot as plt
import torch
from torch import nn, optim, Tensor
from torch.nn import functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils
import numbers
import time


from sklearn.decomposition import PCA

In [ ]:
torch.cuda.manual_seed(5)
torch.manual_seed(5)
np.random.seed(42)
# random.seed(42)

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    #torchvision.transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

train_dataset = torchvision.datasets.MNIST(
    root="~/torch_datasets", train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.MNIST(
    root="~/torch_datasets", train=False, transform=transform, download=True
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=4
)

In [ ]:
class LinearAutoEncoder(nn.Module):
    def __init__(self):
        super(LinearAutoEncoder, self, ).__init__()
        self.layer1 = nn.Linear(in_features=28*28, out_features=128)
#         self.layer2 = nn.Linear(in_features=128, out_features=128)
#         self.layer3 = nn.Linear(in_features=128, out_features=128)
#         self.layer4 = torch.nn.utils.weight_norm(nn.Linear(in_features=128, out_features=28*28))
        self.layer4 = nn.Linear(in_features=128, out_features=28*28)
        
    def forward(self, tensor):
        tensor = tensor.view(-1, 784)
        tensor = self.layer1(tensor)
#         tensor = torch.relu(tensor)
#         tensor = self.layer2(tensor)
#         tensor = torch.relu(tensor)
#         tensor = self.layer3(tensor)
#         tensor = torch.relu(tensor)
        tensor = self.layer4(tensor)
#         tensor = torch.relu(tensor)
        tensor = tensor.view(-1, 1, 28, 28)
        return tensor

In [ ]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

storage = {}
# storage{model_label}[run][epoch]{time_and_loss_label}[data]

epochs = 40
runs = 1
lr_start = 1e-3

lin_label = "linear"
pca_label = "pca"

loss_label = "loss"
time_label = "time"

def train(mdl, label, lr_start, epochs, runs):
    
    # add result dict for each epoch for each run
    storage[label] = [[{} for i in range(epochs)] for j in range(runs)]
    
    # add storage for models
    models = []
    
    for run in range(runs):
        
        # initialize model class and send to device
        model = mdl().to(device)
        
        # initialize new optimizer with model parameters
        optimizer = optim.Adam(model.parameters(), lr=lr_start)
        
        # initialize criterion
        criterion = nn.MSELoss()
            
        # start timing for epochs
        start_time = time.time()

#         scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

        for epoch in range(epochs):

            # initialize the loss
            loss = 0

            for batch_features, _ in train_loader:

                # reshape mini-batch data to [N, 784] matrix
                # load it to the active device
                batch_features = batch_features.to(device)

                # reset the gradients back to zero
                # PyTorch accumulates gradients on subsequent backward passes
                optimizer.zero_grad()

                # compute reconstructions
                outputs = model(batch_features)

                # compute training reconstruction loss
                train_loss = criterion(outputs, batch_features)

                # compute accumulated gradients
                train_loss.backward()

                # perform parameter update based on current gradients
                optimizer.step()

                # add the mini-batch training loss to epoch loss
                loss += train_loss.item()

            # learning rate scheduler
#             scheduler.step()

            # compute the epoch training loss
            loss = loss / len(train_loader)

            # epoch duration
            dur = time.time() - start_time

            # display the epoch training loss
            print("model : {}, run : {}/{}, epoch : {}/{}, loss = {:.6f}, duration = {:.1f}, lr : {}".format(label, run + 1, runs, epoch + 1, epochs, loss, dur, get_lr(optimizer)))

            storage[label][run][epoch][loss_label] = loss
            storage[label][run][epoch][time_label] = dur
            
        # Add to models
        models.append(model)
    
    # return list of models and storage
    return models
        
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
        
lin_models = train(LinearAutoEncoder, lin_label, lr_start, epochs, runs)


In [ ]:
# GET FULL DATA SET IN B WITH SHAPE (60000, 28, 28)
print(train_dataset[59999][0].shape)
test = [train_dataset[i][0] for i in range(len(train_dataset))]
b = torch.Tensor(len(train_dataset), 1, 28, 28)
torch.cat(test, out=b)
print(b.shape)



# # GET FIRST BATCH OF MACHINE
print('\nlinear')
# lin_models[0].float()?
mean_pixels = torch.mean(b, axis=0)
# centered = b - mean_pixels
first_batch = b.unsqueeze(1)[:128,:,:,:]
first_batch = first_batch[:,:,:,:].to(device)

# # RESULT OF LINEAR AE
lin_models[0].float()
lin_result = lin_models[0](first_batch)
lin_models[0].double()



# FIT PCA ON DATASET
print('\nsckit pca')
pca = PCA(n_components=128)
pca.fit(b.view(-1, 784))
eigen_values = pca.explained_variance_
eigen_vectors = pca.components_
projected = pca.transform(b.view(-1, 784))
reconstructed = torch.from_numpy(np.matmul(projected, eigen_vectors)).view(-1, 28, 28)

# MANUAL PCA
print('\nmanual pca')
mean = torch.mean(b.view(-1, 784), axis=0)
cent = b.view(-1, 784) - mean
cov = np.cov(cent.T)
eigvals, eigvecs = np.linalg.eigh(cov)
principal_eigvals = eigvals[np.argsort(-eigvals)[:128]]
principal_eigvecs = eigvecs[:, np.argsort(-eigvals)[:128]].T
proj = principal_eigvecs.dot(cent.T).T
manual_reconstructed = torch.from_numpy(np.matmul(proj, principal_eigvecs)).view(-1, 28, 28)




fig, axes = plt.subplots(4, 5, figsize=(15,10))
for i in range(5):
    axes[0][i].imshow(first_batch[i].cpu().squeeze(0) - mean_pixels, cmap="gray")
    axes[1][i].imshow(lin_result[i].cpu().detach().squeeze(0) - mean_pixels, cmap="gray")
    axes[2][i].imshow(reconstructed[i], cmap="gray")
    axes[3][i].imshow(manual_reconstructed[i], cmap="gray")

axes[0][0].set_ylabel("original")
axes[1][0].set_ylabel("linear")
axes[2][0].set_ylabel("sckit-pca")
axes[3][0].set_ylabel("manual-pca")
plt.show()

In [ ]:
# GET FULL DATA SET IN B WITH SHAPE (60000, 28, 28)
print(train_dataset[59999][0].shape)
test = [train_dataset[i][0] for i in range(len(train_dataset))]
b = torch.Tensor(len(train_dataset), 1, 28, 28)
torch.cat(test, out=b)
print(b.shape)



# # GET FIRST BATCH OF MACHINE
print('\nlinear')
# lin_models[0].float()?
mean_pixels = torch.mean(b, axis=0)
# centered = b - mean_pixels
first_batch = b.unsqueeze(1)[:128,:,:,:]
first_batch = first_batch[:,:,:,:].to(device)

# # RESULT OF LINEAR AE
lin_models[0].float()
lin_result = lin_models[0](first_batch)
lin_models[0].double()



# FIT PCA ON DATASET
print('\nsckit pca')
pca = PCA(n_components=128)
pca.fit(b.view(-1, 784))
eigen_values = pca.explained_variance_
eigen_vectors = pca.components_
projected = pca.transform(b.view(-1, 784))
reconstructed = torch.from_numpy(np.matmul(projected, eigen_vectors)).view(-1, 28, 28)

# MANUAL PCA
print('\nmanual pca')
mean = torch.mean(b.view(-1, 784), axis=0)
cent = b.view(-1, 784) - mean
cov = np.cov(cent.T)
eigvals, eigvecs = np.linalg.eigh(cov)
principal_eigvals = eigvals[np.argsort(-eigvals)[:128]]
principal_eigvecs = eigvecs[:, np.argsort(-eigvals)[:128]].T
proj = principal_eigvecs.dot(cent.T).T
manual_reconstructed = torch.from_numpy(np.matmul(proj, principal_eigvecs)).view(-1, 28, 28)




fig, axes = plt.subplots(4, 5, figsize=(15,10))
for i in range(5):
    axes[0][i].imshow(first_batch[i].cpu().squeeze(0), cmap="gray")
    axes[1][i].imshow(lin_result[i].cpu().detach().squeeze(0), cmap="gray")
    axes[2][i].imshow(reconstructed[i] + mean_pixels, cmap="gray")
    axes[3][i].imshow(manual_reconstructed[i] + mean_pixels, cmap="gray")

axes[0][0].set_ylabel("original")
axes[1][0].set_ylabel("linear")
axes[2][0].set_ylabel("sckit-pca")
axes[3][0].set_ylabel("manual-pca")
plt.show()

In [ ]:
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eigh
# define a matrix
A = array([[1, 3, 2], [6, 4, 1], [5, 6, 5]])
print(A)
# calculate the mean of each column
M = mean(A.T, axis=1)
print(M)
# center columns by subtracting column means
C = A - M
print(C)
# calculate covariance matrix of centered matrix
V = cov(C.T)
print(V)
# eigendecomposition of covariance matrix
values, vectors = eigh(V)
principal_values = values[np.argsort(-values)[:2]]
principal_vectors = vectors[:,np.argsort(-values)[:2]].T
print(principal_vectors)
print(principal_values)
# project data
P = principal_vectors.dot(C.T).T
print(P)

In [ ]:

# Principal Component Analysis
from numpy import array
from sklearn.decomposition import PCA
# define a matrix
A = array([[1, 3, 2], [6, 4, 1], [5, 6, 5]])
print(A)
# create the PCA instance
pca = PCA(2)
# fit on data
pca.fit(A)
# access values and vectors
print(pca.components_)
print(pca.explained_variance_)
# transform data
B = pca.transform(A)
print(B)
